In [ ]:
%pip install requests

In [234]:
from pandas import read_csv, DataFrame
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error
import pandas as pd
import numpy as np
import requests

In [205]:
train = read_csv('andan-2022-autumn-lab-4/train.csv')
test_x = read_csv('andan-2022-autumn-lab-4/test_x.csv')
sample_submission = read_csv('andan-2022-autumn-lab-4/sample_submission.csv')

In [ ]:
cities = set(train.city.values).union(set(test_x.city.values))
importance = {}
i = 0
for city in cities:
    request_city = '+'.join(city.split())
    response = requests.get(f'https://nominatim.openstreetmap.org/search.php?q={request_city}&format=jsonv2').json()
    if (len(response) == 0):
        importance[city] = 0
    else:
        importance[city] = response[0]['importance']
    if i % 100 == 0:
        print(i, city, importance[city])
    i += 1

In [ ]:
imp = importance.copy()
imp

In [288]:
def preprocess(data):
    global imp
    data = data.assign(
        city_importance = data.city.apply(lambda x : imp[x])
    )
    data = data.drop(['floor', 'city', 'osm_city_nearest_name', 'street', 'id'], axis=1)
    data['date'] = pd.to_datetime(data.date, format='%Y-%m-%d')
    data['date'] = (data.date - data.date.min()).dt.days
    data.osm_city_nearest_population.fillna(value=data.osm_city_nearest_population.mode()[0], inplace=True)
    data.reform_house_population_1000.fillna(value=data.reform_house_population_1000.mode()[0], inplace=True)
    data.reform_house_population_500.fillna(value=data.reform_house_population_500.mode()[0], inplace=True)
    data.reform_mean_floor_count_1000.fillna(value=data.reform_mean_floor_count_1000.mean(), inplace=True)
    data.reform_mean_floor_count_500.fillna(value=data.reform_mean_floor_count_500.mean(), inplace=True)
    data.reform_mean_year_building_1000.fillna(value=data.reform_mean_year_building_1000.mean(), inplace=True)
    data.reform_mean_year_building_500.fillna(value=data.reform_mean_year_building_500.mean(), inplace=True)
    one_hot_regions = pd.get_dummies(data['region'])
    data = data.drop(['region'], axis=1).join(one_hot_regions)
    return data

In [301]:
x_train = preprocess(train)
y_train = train.per_square_meter_price
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#    display(abs(x_corr.per_square_meter_price) > 0.05)
x_train = x_train.drop(['per_square_meter_price'], axis=1)

In [302]:
x_train.city_importance

0         0.654271
1         0.694813
2         0.664911
3         0.665946
4         0.718438
            ...   
206469    0.743440
206470    0.718438
206471    0.654271
206472    0.923213
206473    0.734856
Name: city_importance, Length: 206474, dtype: float64

In [303]:
model = LinearRegression()
model.fit(x_train, np.log(y_train))

LinearRegression()

In [304]:
x_test = preprocess(test_x)

In [305]:
y_test = model.predict(x_test)

In [306]:
ans = pd.DataFrame()
ans['per_square_meter_price'] = np.exp(y_test)
ans['id'] = ans.index.values
ans = ans.set_index('id')
ans = ans / 5
ans

,per_square_meter_price
id,
0,23395.131659
1,5474.966895
2,7730.328624
3,9497.484719
4,9711.922234
...,...
68820,4371.100521
68821,8618.570901
68822,16345.908803


In [307]:
ans.to_csv('andan-2022-autumn-lab-4/submission.csv')